## Test Forward Propagation

In [1]:
import numpy as np

In [2]:
import autodiff as ad
x2 = ad.Variable(name = "x2")
x3 = ad.Variable(name = "x3")

y = x2 * x2 + x2 * x3
print(y)

((x2*x2)+(x2*x3))


In [3]:
executor = ad.Executor([y])
y_val = executor.run(feed_dict={
    x2 : 2,
    x3 : 5
})

init val_map: {x2: 2, x3: 5}
topo_order: [x2, (x2*x2), x3, (x2*x3), ((x2*x2)+(x2*x3))]
------------------------------
0 x2
------------------------------
1 (x2*x2)
inputs: 1 [x2, x2]
inputs map: 1 [2, 2]
------------------------------
2 x3
------------------------------
3 (x2*x3)
inputs: 3 [x2, x3]
inputs map: 3 [2, 5]
------------------------------
4 ((x2*x2)+(x2*x3))
inputs: 4 [(x2*x2), (x2*x3)]
inputs map: 4 [4, 10]


In [4]:
print(y_val)

[14]


## Test Gradient of AddOp and MulOp

In [5]:
node_to_output_grads_list = {}
node_to_output_grads_list[y] = [ad.oneslike_op(y)]
node_to_output_grad = {}
reverse_topo_order = list(reversed(ad.find_topo_sort([y])))

print(reverse_topo_order)
print('-'*30)
print(node_to_output_grads_list)
print('-'*30)

[((x2*x2)+(x2*x3)), (x2*x3), x3, (x2*x2), x2]
------------------------------
{((x2*x2)+(x2*x3)): [Oneslike(((x2*x2)+(x2*x3)))]}
------------------------------


In [6]:
def pretty(name, d, indent=0):
    print(name+':')
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):
            pretty(value, indent+1)
        else:
            print('\t' * (indent+1) + str(value))

In [7]:
for _idx, node in enumerate(reverse_topo_order):
    print('-'*30)
    print(_idx, node)
    print('op', node.op)
    print('grad list', node_to_output_grads_list[node])
    grad = ad.sum_node_list(node_to_output_grads_list[node])
    node_to_output_grad[node] = grad
    input_grads = node.op.gradient(node, grad)
    
    print('grad:', grad)
    print('inp_grads:', input_grads)
    print('inp:', node.inputs)
    for _i, inp_node in enumerate(node.inputs):
        node_to_output_grads_list[inp_node] = \
        node_to_output_grads_list[inp_node] + [input_grads[_i]] \
        if inp_node in node_to_output_grads_list.keys()\
        else \
        [input_grads[_i]]
    


grad_node_list = [node_to_output_grad[node] for node in [x2,x3]]        
        
pretty('\n node_to_output_grads_list', node_to_output_grads_list)
pretty('\n node_to_output_grad', node_to_output_grad)

print('-'*30)
print(grad_node_list)

------------------------------
0 ((x2*x2)+(x2*x3))
op <autodiff.AddOp object at 0x7fac1396c208>
grad list [Oneslike(((x2*x2)+(x2*x3)))]
grad: Oneslike(((x2*x2)+(x2*x3)))
inp_grads: [Oneslike(((x2*x2)+(x2*x3))), Oneslike(((x2*x2)+(x2*x3)))]
inp: [(x2*x2), (x2*x3)]
------------------------------
1 (x2*x3)
op <autodiff.MulOp object at 0x7fac1396c240>
grad list [Oneslike(((x2*x2)+(x2*x3)))]
grad: Oneslike(((x2*x2)+(x2*x3)))
inp_grads: [(Oneslike(((x2*x2)+(x2*x3)))*x3), (Oneslike(((x2*x2)+(x2*x3)))*x2)]
inp: [x2, x3]
------------------------------
2 x3
op <autodiff.PlaceholderOp object at 0x7fac1396c320>
grad list [(Oneslike(((x2*x2)+(x2*x3)))*x2)]
grad: (Oneslike(((x2*x2)+(x2*x3)))*x2)
inp_grads: None
inp: []
------------------------------
3 (x2*x2)
op <autodiff.MulOp object at 0x7fac1396c240>
grad list [Oneslike(((x2*x2)+(x2*x3)))]
grad: Oneslike(((x2*x2)+(x2*x3)))
inp_grads: [(Oneslike(((x2*x2)+(x2*x3)))*x2), (Oneslike(((x2*x2)+(x2*x3)))*x2)]
inp: [x2, x2]
------------------------------


In [8]:
executor = ad.Executor(grad_node_list)
executor.run(feed_dict = {x2 : np.ones(1)*2, x3 : np.ones(1)*5})

init val_map: {x2: array([2.]), x3: array([5.])}
topo_order: [x2, (x2*x2), x3, (x2*x3), ((x2*x2)+(x2*x3)), Oneslike(((x2*x2)+(x2*x3))), (Oneslike(((x2*x2)+(x2*x3)))*x3), (Oneslike(((x2*x2)+(x2*x3)))*x2), ((Oneslike(((x2*x2)+(x2*x3)))*x3)+(Oneslike(((x2*x2)+(x2*x3)))*x2)), (Oneslike(((x2*x2)+(x2*x3)))*x2), (((Oneslike(((x2*x2)+(x2*x3)))*x3)+(Oneslike(((x2*x2)+(x2*x3)))*x2))+(Oneslike(((x2*x2)+(x2*x3)))*x2)), (Oneslike(((x2*x2)+(x2*x3)))*x2)]
------------------------------
0 x2
------------------------------
1 (x2*x2)
inputs: 1 [x2, x2]
inputs map: 1 [array([2.]), array([2.])]
------------------------------
2 x3
------------------------------
3 (x2*x3)
inputs: 3 [x2, x3]
inputs map: 3 [array([2.]), array([5.])]
------------------------------
4 ((x2*x2)+(x2*x3))
inputs: 4 [(x2*x2), (x2*x3)]
inputs map: 4 [array([4.]), array([10.])]
------------------------------
5 Oneslike(((x2*x2)+(x2*x3)))
inputs: 5 [((x2*x2)+(x2*x3))]
inputs map: 5 [array([14.])]
------------------------------
6 (Onesl

[array([9.]), array([2.])]